In [13]:
import pandas as pd

"""
0   actual/actual (default) — Number of days in both a period and a year is the actual number of days.
1   30/360 SIA — Year fraction is calculated based on a 360 day year with 30-day months, after applying the following rules: If the first date and the second date are the last day of February, the second date is changed to the 30th. If the first date falls on the 31st or is the last day of February, it is changed to the 30th. If after the preceding test, the first day is the 30th and the second day is the 31st, then the second day is changed to the 30th.
2   actual/360 — Number of days in a period is equal to the actual number of days, however the number of days in a year is 360.
3   actual/365 — Number of days in a period is equal to the actual number of days, however the number of days in a year is 365 (even in a leap year).
4   30/360 PSA — Number of days in every month is set to 30 (including February). If the start date of the period is either the 31st of a month or the last day of February, the start date is set to the 30th, while if the start date is the 30th of a month and the end date is the 31st, the end date is set to the 30th. The number of days in a year is 360.
5   30/360 ISDA — Number of days in every month is set to 30, except for February where it is the actual number of days. If the start date of the period is the 31st of a month, the start date is set to the 30th while if the start date is the 30th of a month and the end date is the 31st, the end date is set to the 30th. The number of days in a year is 360.
6   30E /360 — Number of days in every month is set to 30 except for February where it is equal to the actual number of days. If the start date or the end date of the period is the 31st of a month, that date is set to the 30th. The number of days in a year is 360.
7   actual/365 Japanese — Number of days in a period is equal to the actual number of days, except for leap days (29th February) which are ignored. The number of days in a year is 365 (even in a leap year).
https://www.mathworks.com/help/fininst/day-count-basis.html
"""

'\n0   actual/actual (default) — Number of days in both a period and a year is the actual number of days.\n1   30/360 SIA — Year fraction is calculated based on a 360 day year with 30-day months, after applying the following rules: If the first date and the second date are the last day of February, the second date is changed to the 30th. If the first date falls on the 31st or is the last day of February, it is changed to the 30th. If after the preceding test, the first day is the 30th and the second day is the 31st, then the second day is changed to the 30th.\n2   actual/360 — Number of days in a period is equal to the actual number of days, however the number of days in a year is 360.\n3   actual/365 — Number of days in a period is equal to the actual number of days, however the number of days in a year is 365 (even in a leap year).\n4   30/360 PSA — Number of days in every month is set to 30 (including February). If the start date of the period is either the 31st of a month or the la

In [14]:
def day_count(t1, t2, basis):
    t1 = pd.to_datetime(t1)
    t2 = pd.to_datetime(t2)

    def is_leap(year):
        return (year % 400 == 0) or (year % 100 != 0 and year % 4 == 0)

    if t1 > t2:
        print("t1 is greater than t2")
        return
    match basis:
        case 0:  # actual/actual
            return (t2 - t1).days
        case 1:  # 30/360 (SIA)
            d1, d2 = t1.day, t2.day
            if t1.day == 31:
                d1 = 30
            if d1 == 30 and t2.day == 31:
                d2 = 30
            if t1.month == 2 and t1.day == t1.days_in_month:
                d1 = 30
                if t2.month == 2 and t2.day == t2.days_in_month:
                    d2 = 30
            total_days = (
                360 * (t2.year - t1.year) + 30 * (t2.month - t1.month) + d2 - d1
            )
            return total_days
        case 2:  # actual/360
            return (t2 - t1).days
        case 3:  # actual/365
            return (t2 - t1).days
        case 4:  # 30/360 (PSA)
            d1, d2 = t1.day, t2.day
            if t1.day == 31 or (t1.month == 2 and t1.day == t1.days_in_month):
                d1 = 30
            if d1 == 30 and t2.day == 31:
                d2 = 30
            total_days = (
                360 * (t2.year - t1.year) + 30 * (t2.month - t1.month) + d2 - d1
            )
            return total_days
        case 5:  # 30/360 (ISDA)
            d1, d2 = t1.day, t2.day
            if t1.day == 31 or (
                t1.day == 31 or (t1.month == 2 and t1.day == t1.days_in_month)
            ):
                d1 = 30
            if d1 == 30 and t2.day == 31:
                d2 = 30
            total_days = (
                360 * (t2.year - t1.year) + 30 * (t2.month - t1.month) + d2 - d1
            )
            return total_days
        case 6:  # 30/360 (European)
            d1, d2 = t1.day, t2.day
            if t1.day == 31 or (t1.month == 2 and t1.day == t1.days_in_month):
                d1 = 30
            if t2.day == 31:
                d2 = 30
            total_days = (
                360 * (t2.year - t1.year) + 30 * (t2.month - t1.month) + d2 - d1
            )
            return total_days
        case 7:  # Actual/365 (Japanese)
            total_days = (t2 - t1).days
            leap_days = 0
            for year in range(t1.year, t2.year + 1):
                if (year % 400 == 0) or (year % 100 != 0 and year % 4 == 0):
                    if t1 <= pd.to_datetime(f"{year}-02-29") < t2:
                        leap_days += 1
            total_days -= leap_days
            return total_days

In [15]:
def year_count(t1, t2, basis):

    t1 = pd.to_datetime(t1)
    t2 = pd.to_datetime(t2)

    def is_leap(year):
        return (year % 400 == 0) or (year % 100 != 0 and year % 4 == 0)

    if t1 > t2:
        print("t1 is greater than t2")
        return
    match basis:
        case 0:  # actual/actual
            total = 0.0
            temp = t1
            while temp < t2:
                end_date = pd.to_datetime(f"{temp.year+1}-01-01")
                end_date = min(end_date, t2)
                temp_days = (end_date - temp).days
                year_days = 366 if is_leap(temp.year) else 365
                total += temp_days / year_days
                temp = end_date
            return total
        case 1:  # 30/360 (SIA)
            d1, d2 = t1.day, t2.day
            if t1.day == 31:
                d1 = 30
            if d1 == 30 and t2.day == 31:
                d2 = 30
            if t1.month == 2 and t1.day == t1.days_in_month:
                d1 = 30
                if t2.month == 2 and t2.day == t2.days_in_month:
                    d2 = 30
            total_days = (
                360 * (t2.year - t1.year) + 30 * (t2.month - t1.month) + d2 - d1
            )
            return total_days / 360.0
        case 2:  # actual/360
            total_days = (t2 - t1).days
            return total_days / 360.0
        case 3:  # actual/365
            total_days = (t2 - t1).days
            return total_days / 365
        case 4:  # 30/360 (PSA)
            d1, d2 = t1.day, t2.day
            if t1.day == 31 or (t1.month == 2 and t1.day == t1.days_in_month):
                d1 = 30
            if d1 == 30 and t2.day == 31:
                d2 = 30
            total_days = (
                360 * (t2.year - t1.year) + 30 * (t2.month - t1.month) + d2 - d1
            )
            return total_days / 360.0
        case 5:  # 30/360 (ISDA)
            d1, d2 = t1.day, t2.day
            if t1.day == 31 or (t1.month == 2 and t1.day == t1.days_in_month):
                d1 = 30
            if d1 == 30 and t2.day == 31:
                d2 = 30
            total_days = (
                360 * (t2.year - t1.year) + 30 * (t2.month - t1.month) + d2 - d1
            )
            return total_days / 360.0
        case 6:  # 30/360 (European)
            d1, d2 = t1.day, t2.day
            if t1.day == 31 or (t1.month == 2 and t1.day == t1.days_in_month):
                d1 = 30
            if t2.day == 31:
                d2 = 30
            total_days = (
                360 * (t2.year - t1.year) + 30 * (t2.month - t1.month) + d2 - d1
            )
            return total_days / 360.0
        case 7:  # Actual/365 (Japanese)
            total_days = (t2 - t1).days
            leap_days = 0
            for year in range(t1.year, t2.year + 1):
                if (year % 400 == 0) or (year % 100 != 0 and year % 4 == 0):
                    if t1 <= pd.to_datetime(f"{year}-02-29") < t2:
                        leap_days += 1
            total_days -= leap_days
            return total_days / 365.0

In [26]:
t1 = pd.to_datetime("2024-02-29")
t2 = pd.to_datetime("2024-03-01")

In [27]:
for i in range(8):
    print("Basis ", i, ": ", day_count(t1, t2, basis=i))

Basis  0 :  1
Basis  1 :  1
Basis  2 :  1
Basis  3 :  1
Basis  4 :  1
Basis  5 :  1
Basis  6 :  1
Basis  7 :  0


In [28]:
for i in range(8):
    print("Basis ", i, ": ", year_count(t1, t2, basis=i))

Basis  0 :  0.00273224043715847
Basis  1 :  0.002777777777777778
Basis  2 :  0.002777777777777778
Basis  3 :  0.0027397260273972603
Basis  4 :  0.002777777777777778
Basis  5 :  0.002777777777777778
Basis  6 :  0.002777777777777778
Basis  7 :  0.0
